https://github.com/MuneebUH?tab=repositories

In [ ]:
!pip -q install googlemaps
!pip -q install requests


  Preparing metadata (setup.py) ... done


In [ ]:
import requests
import googlemaps
import json
from datetime import timedelta  # Import timedelta for formatting

In [ ]:
API_KEY = ""
gmaps = googlemaps.Client(API_KEY)

In [ ]:
# Function to get latitude & longitude from place name
def get_coordinates(place_name):
    geocode_result = gmaps.geocode(place_name)
    if geocode_result:
        location = geocode_result[0]['geometry']['location']
        return {"latitude": location["lat"], "longitude": location["lng"]}
    else:
        print(f"❌ Error: Unable to find location for '{place_name}'.")
        return None

# Function to format duration using timedelta
def format_duration(seconds):
    duration = timedelta(seconds=seconds)  # Convert seconds to timedelta
    days = duration.days
    hours, remainder = divmod(duration.seconds, 3600)
    minutes = remainder // 60

    return f"{days} days {hours} hrs {minutes} min" if days > 0 else f"{hours} hrs {minutes} min"

# Get user input for origin and destination
print("\n🚗 Enter your starting point and destination to calculate distance & travel time.\n")
origin_input = input("📍 Enter your starting location (city name or coordinates, e.g., '33.6844,73.0479'): ")
destination_input = input("🎯 Enter your destination (city name or coordinates, e.g., '31.5497,74.3436'): ")

# Determine if input is a city name or coordinates
def parse_location_input(location):
    if "," in location:
        try:
            lat, lng = map(float, location.split(","))
            return {"latitude": lat, "longitude": lng}
        except ValueError:
            print(f"❌ Invalid coordinates: {location}")
            return None
    else:
        return get_coordinates(location)

# Get coordinates
origin_coordinates = parse_location_input(origin_input)
destination_coordinates = parse_location_input(destination_input)

# Ensure valid coordinates were found
if not origin_coordinates or not destination_coordinates:
    print("⚠️ Invalid input. Please enter valid city names or coordinates.")
    exit()

# Routes API endpoint
URL = "https://routes.googleapis.com/distanceMatrix/v2:computeRouteMatrix"

# Request payload
payload = {
    "origins": [{"waypoint": {"location": {"latLng": origin_coordinates}}}],
    "destinations": [{"waypoint": {"location": {"latLng": destination_coordinates}}}],
    "travelMode": "DRIVE",
    "routingPreference": "TRAFFIC_AWARE"
}

# Request headers
headers = {
    "Content-Type": "application/json",
    "X-Goog-Api-Key": API_KEY,
    "X-Goog-FieldMask": "originIndex,destinationIndex,duration,distanceMeters,status,condition"
}

# Make API request
response = requests.post(URL, headers=headers, json=payload)

# Process response
if response.status_code == 200:
    try:
        routes = response.json()
        for route in routes:
            distance_meters = route.get("distanceMeters", 0)
            duration_seconds = int(route.get("duration", "0s").replace("s", ""))

            # Convert distance to kilometers
            distance_km = distance_meters / 1000

            # Format duration using timedelta
            formatted_duration = format_duration(duration_seconds)

            print(f"\n✅ Route from {origin_input} to {destination_input}:")
            print(f"   📏 Distance: {distance_km:.2f} km")
            print(f"   ⏳ Duration: {formatted_duration}\n")
    except json.JSONDecodeError:
        print("⚠️ Error: Unable to parse response JSON.")
else:
    print(f"❌ API Error: {response.status_code} - {response.text}")



🚗 Enter your starting point and destination to calculate distance & travel time.

📍 Enter your starting location (city name or coordinates, e.g., '33.6844,73.0479'): Lahore
🎯 Enter your destination (city name or coordinates, e.g., '31.5497,74.3436'): Karachi

✅ Route from Lahore to Karachi:
   📏 Distance: 1221.82 km
   ⏳ Duration: 14 hrs 32 min

